# Imports

In [6]:
import numpy as np
import pandas as pd
import codecs
import matplotlib.pyplot as plt

# Reading the data

In [7]:
def strip_line(line):
    """
    A function that removes non-alpha characters, single char words, and non hebrew words
    It also removes duplicate letters at the end of a word
    Input:: line
    type:: string
    output:: X,y
    type:: string
    """
    y = int(line[-1])
    x = [word for word in line.split() if word.isalpha() and len(word)>1]
    for word in x.copy():

        for char in word:
            if char not in 'אבגדהוזחטיכלמנסעפצקרשתךםןףץ':
                x.remove(word)
                break
    
    for i, word in enumerate(x):
        word_cp = str(word)
        last_chr = word[-1]

        for j, char in enumerate(word_cp[::-1]):
            if char == last_chr:
                if j != 0:
                    word = word[:-1]
            else:
                last_chr = ''

        x.remove(word_cp)
        x.insert(i, word)
    x = ' '.join(x)
        
    return x, y

In [8]:
# docs is the final dataset that will transform using TF-IDF
def make_X_y(data):
    docs = []
    y = []
    for line in data:
        x, y0 = strip_line(line)
        docs.append(x)
        y.append(y0)
    return docs, np.array(y)

In [10]:
data_train = codecs.open('./data/token_train.tsv', 'r', 'utf-8').read().splitlines()
data_test = codecs.open('./data/token_test.tsv', 'r', 'utf-8').read().splitlines()
X_docs_train, y_train = make_X_y(data_train)
X_docs_test, y_test = make_X_y(data_test)

In [11]:
y_test.shape

(2560,)

# Perform TF-IDF transformation using sklearn

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf=True)#, min_df=10, max_df=100)

In [13]:
X_train = tfidf_vectorizer.fit_transform(X_docs_train).toarray()
X_test = tfidf_vectorizer.transform(X_docs_test).toarray()

In [14]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(10244, 25799) (10244,)
(2560, 25799) (2560,)


# Training the classifier

In [15]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

# Making a confusion matrix

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred_train = classifier.predict(X_train)
cm_train = confusion_matrix(y_train, y_pred_train)
print(cm_train)
accuracy_score(y_train, y_pred_train)

[[6688  125    1]
 [ 157 2975    0]
 [ 138  100   60]]


0.9491409605622804

In [18]:
y_pred_test = classifier.predict(X_test)
cm_test = confusion_matrix(y_test, y_pred_test)
print(cm_test)
accuracy_score(y_test, y_pred_test)

[[1634   63    1]
 [ 117  673    0]
 [  37   27    8]]


0.904296875

In [19]:
manual_check = ['הנשיא ריבלין הוא אידיוט לא מאמין שהצבעתי לך'
         ,'הנשיא הזה הוא בחירה גרועה'
         ,'אני אוהב אותך נשיא יקר'
         ,'זה פשוט חלום אחלה של נזשיא בהצלחה מרובה'
         ,'אכזבה ממישהו שציפיתי שיגיד דעה חדה וברורה מתבייש בך רובי , אכזבת אותי'
         ,'תומך במחבלים שמאלני'
         ]
X_check = tfidf_vectorizer.transform(manual_check).toarray()
y_check = classifier.predict(X_check)

In [21]:
translation_dict = {0: 'Positive', 1: 'negative', 2: 'unknown'}
for line, y in zip(manual_check, y_check):
    print(line, translation_dict[y])
    print('----')

הנשיא ריבלין הוא אידיוט לא מאמין שהצבעתי לך negative
----
הנשיא הזה הוא בחירה גרועה Positive
----
אני אוהב אותך נשיא יקר Positive
----
זה פשוט חלום אחלה של נזשיא בהצלחה מרובה Positive
----
אכזבה ממישהו שציפיתי שיגיד דעה חדה וברורה מתבייש בך רובי , אכזבת אותי negative
----
תומך במחבלים שמאלני negative
----
